# Task: Wifi Positioning with RSSI Fingerprints
You will get a dataset that was gathered by the Minodes team to enable wifi positioning. In this store we installed 145 of our sensors, also called **nodes**. 

As you walk through the store your phone searches for available wifi networks, by sending probe requests (see 802.11 standard). A part of this probe request is the received signal strength indication (**RSSI**), which has a range between 0db (very strong signal) and -100db (very weak signal). Depending on the distance of your phone to a sensor, the **RSSI** has a different value. In an ideal case one probe request would be perceived by all nodes of the store with different signal strength. In our data processing we collect all this probe requests and aggregate them, so that we have all **RSSI** values for all nodes of one probe request available:

A data set might look like this:


|  ID             | RSSI value node 1 | RSSI value node 2  | .... | RSSI value node n  | fr_zone_id |
| -------------   |-------------      | -----              | -----| -------------------| ------- |
| 1               |-70                | -70                | .... |  -70               | 1       |
| 2               |-55                | -60                | .... |  -45               | 2       |
| 2               |-60                | -60                | .... |  -45               | 2       |
| ...             |.......            | ....               | .... | ....               | ..      |

Each line of this the table is called a fingerprint, which gives us an indication about the potential location of a person.

To locate a person within the store, we separate the store into zones representing compartments of a store. For each of this **zones** we collect reference data, which is stored in the file *training_set.csv* of this exercise.

The goal of this task is to use machine learning techniques to predict the correct zones, based on **RSSI** fingerprints. At the end of this task you need to predict the zones of unkown **RSSI** fingerprints stored in the file *test_without_target.csv*. 

Since we at Minodes love to code, we have done some preparation work, so you can focus on the machine learning.

In case you have any questions please contact **alexander.mueller@minodes.com**

## Remarks

* Please use python 3.x
* I suggest using an anaconda python distribution https://www.continuum.io/downloads
* The code is tested in python 3.4
* Please use pre-existing libraries

## Requirements
The current versions of:
* pandas 
* numpy

## Some imports and a preprocessing routine

In [ ]:
import ast
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model,svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

def preprocess_x_values(x_raw_values):
    """
    A simple preprocessing routine returning a feature vector for the specific fingerprint. 
    """

    v = DictVectorizer(sparse=False)
    
    return v.fit_transform(x_raw_values)

def compute_accuracy(Y,predY):
    
    accuracy = np.sum(Y==predY)/predY.shape[0] # Compute accuracy
    
    return accuracy

### Loading the data

In [ ]:
training_set = pd.read_csv('training_set.csv')

In [ ]:
# Load training data

# Parse dictionary of features
training_set['values'] =training_set['values'].apply(lambda x:  ast.literal_eval(x))

# preparing x and y sets 
# x are the features of the set
# y is the class to be predicted
x_raw = training_set['values']
y = training_set['fr_zone_id']

# preprocess the node dictionary to get feature vectors
x_features= preprocess_x_values(x_raw)

# Add column of 1's as additional feature to the dataset
ones = np.ones((x_features.shape[0],1))
x_features = np.concatenate((x_features,ones),axis=1)

In [ ]:
x_features.shape

In [ ]:
x_features

### Implementation

Think about how you validate your model (based on accuracy), what classifier you want to use and how in the end you want to scorce the test set.

You need to add only a couple of lines of code to have a basic solution. Do this first and then iterate on it.

In [ ]:
#  please start here

# Uncomment a model you wish to train

# Fit a linear regression model
#model = linear_model.LinearRegression()
#model.fit(x_features,y)

# Fit a linear regression model with L2 regularizer
#model = linear_model.Ridge()
#model.fit(x_features,y)

# Fit a logistic regression model
model = linear_model.LogisticRegression()
model.fit(x_features,y.values)

# Fit a SVM model
#model = svm.SVC()
#model.fit(x_features,y)

# Use kNN algorithm
#k = 10 # change hyperparameter k here
#model = KNeighborsClassifier(n_neighbors=k)
#model.fit(x_features,y)

In [ ]:
# Computing accuracy on training data
predY_train = model.predict(x_features)
predY_train = np.rint(predY_train) # Round off predictions to nearest integer, useful for linear/ridge regression

# Compute accuracy
accuracy_train = compute_accuracy(y,predY_train) 
print(accuracy_train)

# Compute confusion matrix
confusion_matrix(y,predY_train)

### Predict and unkown dataset

In [ ]:
test_set = pd.read_csv('test_without_target.csv',index_col='id')
test_set['values'] = test_set['values'].apply(lambda x:  ast.literal_eval(x))
x_test = preprocess_x_values(test_set['values'])

# Add column of 1's as additional feature to the dataset
ones = np.ones((x_test.shape[0],1))
x_test = np.concatenate((x_test,ones),axis=1)

In [ ]:
# Prediction on test data
predY_test = model.predict(x_test)
predY_test = np.rint(predY_test) # Round off predictions to nearest integer, useful for linear/ridge regression

# Compute accuracy on test data (Here y_test are the ground truth labels)
accuracy_test = compute_accuracy(y_test,predY_test)
print(accuracy_test)

# Compute confusion matrix (Here y_test are the ground truth labels)
confusion_matrix(y_test,predY_test)

### Submit your solution

Please zip the complete folder with your solution and send it back to **alexander.mueller@minodes.com**. We will review it as soon as possible and will come back to you!